### SberHackaton
### team WhyNot
we need to predict the next purchase made by customer


1. Download the datasets and observe

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from numpy.linalg import inv
import random
import pandas as pd

### Let's analyze orders dataset

In [2]:

orders = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_1_orders.csv')
city= pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/tab_6_city.csv')

orders.sample(5)

,user_id,order_id,order_created_time,retailer,store_id,platform
74352,156019,12322555,2020-06-20 13:55:10,Ашан,92,undefined
1163344,1532891,11633771,2020-06-08 14:49:42,METRO,107,app
1337301,1672938,6730882,2020-03-03 17:37:56,Лента,145,app
504038,861453,9824324,2020-05-10 10:43:25,Лента,218,app
1838704,2100280,14819920,2020-08-10 22:03:09,Лента,130,app


In [3]:
city.sample(5)

,store_id,city_name
59,117,Новосибирск
632,504,Москва
469,804,Екатеринбург
402,413,Ростов-на-Дону
452,707,Челябинск


In [4]:
city.shape

(788, 2)

In [5]:
orders.shape

(2461958, 6)

In [6]:
orders_c = pd.merge(orders,city,on = 'store_id')
orders_c.sample(3)

,user_id,order_id,order_created_time,retailer,store_id,platform,city_name
701180,1963978,11917987,2020-06-13 15:13:16,METRO,93,app,Московская Область
2456195,3147314,15377980,2020-08-21 15:37:21,Лента,405,app,Майкоп
2273282,1847074,12163663,2020-06-17 15:27:47,Ашан,251,app,Краснодар


### here we can see different retailers in different cities, so if a person from Абакан there is only point to predict that he wil buy something in "Аллея"

In [7]:
city_ratailers = orders_c.groupby(['city_name'])['retailer'].value_counts()
city_ratailers

city_name           retailer            
 Каменск-Уральский  Лента                     49
Абакан              Аллея                    668
Альметьевск         Лента                   1904
Анадырь             ТЦ "Новомариинский"       46
Архангельск         METRO                   1962
                                            ... 
Якутск              УДАЧНАЯ ПОКУПКА          235
                    МаксМаркет               231
Ярославль           METRO                   9515
                    Лента                   7717
                    Ашан                    5305
Name: retailer, Length: 246, dtype: int64

In [8]:
#let's calculate some additional info
print('number of users in the dataset : ' ,orders_c.user_id.nunique()) 

number of users in the dataset :  753572


### **we can see that number of users are less than number of purshases in the dataset so some customers made more than 1 purchase** 

### **let's find users who made more purchases than others**

In [9]:
orders_per_customers = orders_c.groupby(['user_id'])['order_id'].count()
orders_per_customers

user_id
72         1
83         1
142        1
187        1
224        1
          ..
3432957    1
3432966    1
3432977    1
3433000    1
3433023    1
Name: order_id, Length: 753572, dtype: int64

In [10]:
orders_per_customers[orders_per_customers > 1]
#there are a lot of people whp made more than one purshase

user_id
260         2
353         8
400         6
525         3
576        10
           ..
3424582     2
3427265     2
3427304     2
3432418     2
3432861     2
Name: order_id, Length: 355359, dtype: int64

In [11]:
orders_per_customers[orders_per_customers > 15] 
#here we have people who made a lot of purshases

user_id
766        25
1578       27
1724       25
2268       30
2693       18
           ..
2987260    26
3000696    19
3010057    23
3058685    52
3116617    16
Name: order_id, Length: 24406, dtype: int64

Let't find out who are making more purchases than others

In [12]:
orders_per_customers[orders_per_customers > 100] #there is a list of people who made more purhases that others

user_id
144291     106
224747     196
268890     101
300421     185
344968     205
351997     144
450083     115
507109     101
620405     143
863370     107
888699     102
913694     133
958080     152
1028618    105
1093518    140
1167872    110
1225517    106
1478258    113
1533234    107
2253724    108
2325790    102
Name: order_id, dtype: int64

In [13]:
orders_per_customers[orders_per_customers == orders_per_customers.max()]  #there is a record

user_id
344968    205
Name: order_id, dtype: int64

In [14]:
print('number of retailers is : ',orders_c.retailer.nunique())

number of retailers is :  61


We can analyze how people bahave in different retailers

In [15]:
retailer_total = orders_c.groupby(['retailer'])['order_id'].count()
#here we have number of purshases in different retailers
retailer_total

retailer
220 ВОЛЬТ             33
BILLA               2735
Gipfel                11
Home Market           17
METRO            1369726
                  ...   
Улыбка Радуги        122
Фреш25               132
Хороший              546
Шан                   59
ЮгОпт                 14
Name: order_id, Length: 61, dtype: int64

Let's find 3 most popular retailers for customers

In [16]:
retailer_total[retailer_total == retailer_total.max()] 

retailer
METRO    1369726
Name: order_id, dtype: int64

So top 1 retailer is METRO

In [17]:
retailer_total[retailer_total > 100000 ] 

retailer
METRO    1369726
Ашан      401004
Лента     653387
Name: order_id, dtype: int64

TOP 3 retailers are : Metro, Ашан, Лента 

bacically product markets are more popular than any other retailers

In [18]:
print('number of platforms is : ',orders_c.platform.nunique())

number of platforms is :  3


### **We have only 3 platforms so we can analyze how many customers made their purchases there**

In [19]:
orders_c.groupby(['platform'])['user_id'].count()

platform
app          1872404
undefined      76536
web           513018
Name: user_id, dtype: int64

### **We can clearly see that purhases in the app are more preferable than purchases in web, also there is 76536 rows with underfined platform **

//------------------------------------------------------------------------------------------------------------------------//

### Let's analyze products dataset

In [20]:
products = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/sbermarket_tab_2_1/tab_2_products_2020-01-01.csv')
products.sample(5)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id
129581,316696,5574989,29221821,152.190002,1,0.000000,Крыло цыпленка-бройлера Петруха замороженное ~...,4593001,Петруха,122.0,119
180527,427950,5326601,28813007,88.989998,1,40.009998,Маслины Iberica крупные без косточки,2005,Iberica,626.0,67
173166,396816,5482320,28347679,71.889999,1,0.000000,Ежевика Fine Life быстрозамороженная,37277,Fine Life,592.0,90
241184,693530,5535585,28362949,87.190002,1,0.000000,Фасоль Мистраль белая 450 г,198863,Мистраль,580.0,9
99154,265401,5560407,29021435,136.889999,1,0.000000,Желудки куриные замороженные,8028507,Без бренда,125.0,119


### **there are some outliers in discount column (negative values) let's filter outliers (negative discount will be equal to zero)**

In [21]:
# products['discount_real'] = (products.discount >= 0)

# products[products.discount_real == False].discount == 0
products[ (products.discount < 0)].discount = 0


C:\Users\zvere\anaconda3\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
products

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id
0,525,5354800,28381452,116.000000,4,0.000000,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок,101.0,99
1,525,5354800,29242911,56.990002,2,13.080000,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом,596.0,51
2,525,5354800,29242919,64.989998,2,40.020000,Тортилья Delicados оригинальная пшеничная,22107,Delicados,596.0,51
3,525,5354800,29243785,44.990002,6,0.000000,Вода питьевая минеральная BonAqua газированная...,2530,BonAqua,77.0,74
4,525,5354800,29244246,65.900002,6,30.990000,"Газированный напиток Mirinda Апельсин 1,5 л",3818486,Mirinda,76.0,74
...,...,...,...,...,...,...,...,...,...,...,...
303662,725964,5652846,29327078,52.639999,2,23.780001,Черная смородина Fine life быстрозамороженная,94654,Fine Life,592.0,90
303663,725964,5652846,29327108,29.459999,1,39.400002,Помидоры черри красные 250 г,687,Без бренда,85.0,84
303664,725964,5652846,29327109,29.480000,1,29.580000,Помидоры черри сливовидные 250 г,4830888,Без бренда,85.0,84
303665,725964,5652846,29327192,83.220001,1,21.209999,Сыр Bonfesto Моцарелла mini 45% 100 г бзмж,26739,Bonfesto,38.0,35


In [23]:
products.shape

(303667, 11)

In [24]:
products.line_item_id.nunique()

303667

In [25]:
products.order_id.nunique()

12154

In [26]:
products.user_id.nunique()

10580

### Let's analyze how many items in every order and calculate the price for every order
but first we need to create a new column **cost_real** what will include the price ( taking the discount into account) multiplied by quantity 

In [27]:
products['cost_real'] = (products.price - products.discount) * products.quantity
products.head(3)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real
0,525,5354800,28381452,116.000000,4,0.00,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок,101.0,99,464.000000
1,525,5354800,29242911,56.990002,2,13.08,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом,596.0,51,87.820004
2,525,5354800,29242919,64.989998,2,40.02,Тортилья Delicados оригинальная пшеничная,22107,Delicados,596.0,51,49.939995


In [28]:
print('the number of unique products names is', products.product_name.nunique())

the number of unique products names is 26763


In [29]:
print('the number of unique customers is', products.user_id.nunique())

the number of unique customers is 10580


### **Now when we know the number of products and cost we can determine the most expensive products**

In [30]:
products[products.cost_real == products.cost_real.max()] ## the most expensive 

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real
45062,137924,5631897,29129595,159962.046875,1,39.490002,Голень индейки Индилайт без кости охлажденная ...,3818829,Индилайт,122.0,119,159922.556873


In [31]:
most_expensive_product_cost = products.groupby(['product_name'])['price'].max().max()
most_expensive_product_cost

159962.046875

### **We can determine the most expensive products and we can make predictions that people who can buy such an espensive products will buy something in the same field with the same price**

In [32]:
products[(products.price < 100000) & (products.price > 1000)].head(3) #expensive products with high price

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real
355,3225,5350970,28436606,1189.010010,1,510.0,Креветки Бухта изобилия королевская 50/70,5368,Agama,118.0,112,679.010010
453,3674,5307004,28931590,1923.979980,1,0.0,Глазной мускул говяжий Заречное охлажденный ~2...,5214742,Заречное,120.0,119,1923.979980
454,3674,5307004,28932555,2150.189941,1,0.0,Лопатка говяжья Мираторг без кости замороженна...,5063231,Мираторг,121.0,119,2150.189941


### **ALso we have noticed that there are some purchases with quantity more than 1, that means that some people buy the same product more than once. We can analyze it and it will help our model to make predictions for these people better : because if they buy this particular product often there is more chance that they will buy it again**

In [33]:
repeat_customers = products[products.quantity >10]

products['repeated'] = (products.quantity >= 10)
products.groupby(['repeated'])['user_id'].count()

repeated
False    297949
True       5718
Name: user_id, dtype: int64

### **We need to create a column with a number of purshaces for every customer**

In [34]:
number_purchases = products.groupby(['user_id'])['order_id'].count()
products1 = pd.merge(products,number_purchases,on = 'user_id')
products1 = products1.rename(columns = {'order_id_y':'num_purchases', 'order_id_x' :'order_id'})
products = products1
products.num_purchases.nunique()


147

### **there are some columns with quantity = 1 but still with high cost_real. That means that these people buy something in huge portion for once, probably it is people who have their oun markets or cafe. Let't determine them**

In [35]:
# opt_customers = products[(products.cost_real > 5000) & (products.quantity < 5)] 

products['technics'] = (products.cost_real > 5000) & (products.num_purchases < 5) 
products.groupby(['technics'])['user_id'].count()

technics
False    303647
True         20
Name: user_id, dtype: int64

###  **Let's add another bool column "Prefer_Discounts"** 

In [36]:
discounts_sum = products.groupby(['user_id'],as_index = False)['discount'].sum()
discounts1= pd.merge(products,discounts_sum,on = 'user_id')
discounts1 = discounts1.rename(columns = {'discount_x':'discount', 'discount_y' :'discount_total'})
products = discounts1


In [37]:
products

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real,repeated,num_purchases,technics,discount_total
0,525,5354800,28381452,116.000000,4,0.000000,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок,101.0,99,464.000000,False,14,False,630.080004
1,525,5354800,29242911,56.990002,2,13.080000,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом,596.0,51,87.820004,False,14,False,630.080004
2,525,5354800,29242919,64.989998,2,40.020000,Тортилья Delicados оригинальная пшеничная,22107,Delicados,596.0,51,49.939995,False,14,False,630.080004
3,525,5354800,29243785,44.990002,6,0.000000,Вода питьевая минеральная BonAqua газированная...,2530,BonAqua,77.0,74,269.940010,False,14,False,630.080004
4,525,5354800,29244246,65.900002,6,30.990000,"Газированный напиток Mirinda Апельсин 1,5 л",3818486,Mirinda,76.0,74,209.460011,False,14,False,630.080004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303662,725964,5652846,29327078,52.639999,2,23.780001,Черная смородина Fine life быстрозамороженная,94654,Fine Life,592.0,90,57.719997,False,6,False,909.529999
303663,725964,5652846,29327108,29.459999,1,39.400002,Помидоры черри красные 250 г,687,Без бренда,85.0,84,-9.940002,False,6,False,909.529999
303664,725964,5652846,29327109,29.480000,1,29.580000,Помидоры черри сливовидные 250 г,4830888,Без бренда,85.0,84,-0.100000,False,6,False,909.529999
303665,725964,5652846,29327192,83.220001,1,21.209999,Сыр Bonfesto Моцарелла mini 45% 100 г бзмж,26739,Bonfesto,38.0,35,62.010002,False,6,False,909.529999


In [38]:
products['prefer_discounts'] = (products.discount_total > products.discount_total.mean())


In [39]:
products.prefer_discounts.value_counts()

False    199140
True     104527
Name: prefer_discounts, dtype: int64

In [40]:
products.sample(5)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real,repeated,num_purchases,technics,discount_total,prefer_discounts
250922,699668,5555460,28495689,64.900002,2,0.0,Огурец длинноплодный Мое Лето гладкий 300 г,12792,Мое лето,85.0,84,129.800003,False,12,False,442.579998,False
264114,705396,5627128,29112694,99.000000,1,0.0,Батарейки Panasonic Alkaline Power AAA 4 шт,104809,Panasonic,420.0,418,99.000000,False,50,False,431.509992,False
208672,586036,5587402,28742184,142.830002,1,0.0,Крыло куриное с кожей Куриное Царство Целое за...,5164,Куриное Царство,122.0,119,142.830002,False,26,False,151.789997,False
219076,629561,5391078,29046607,87.290001,1,0.0,NaN,0,NaN,NaN,0,87.290001,False,16,False,138.800003,False
54638,161064,5461128,28962768,164.000000,1,0.0,Жидкое мыло Palmolive Натурэль Интенсивное увл...,25168,Palmolive,234.0,232,164.000000,False,42,False,981.519998,False


### **There are also families who have big check, they buy a lot of products for one time. We can find them. Now when we have number of items per user we can detect a families. They buy a lot of products for once so we can group them according do order_id and high number of items in order**

In [41]:
categories= pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_3_categories.csv')
categories.sample(5)


,id,name,parent_id
311,312,Подставки и прихватки,303
137,138,Детские товары,0
214,215,Наборы для бритья,208
726,727,Уход за стеклами,504
34,35,Сыры,1


In [42]:
user_profiles = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_4_user_profiles.csv')
user_profiles.sample(5)


,user_id,gender,bdate
132360,2619553,NaN,NaN
167530,2772130,NaN,NaN
53904,968749,female,1974-11-24
205675,2871061,NaN,NaN
80978,2456287,NaN,NaN


In [43]:
product_properties = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_5_product_properties.csv')
product_properties.sample(5)
                                 

,product_id,property_name,property_value
6302835,13341365,carbohydrate,"10,5 г"
2777920,7468304,Содержание калия,500 мг/л
2481588,7384341,Состав набора,"Помидоры, Огурцы"
2669308,7455525,Состав,"Филе куриное, вода питьевая, соль пищевая, чес..."
4194897,8138136,"Вес упаковки, кг",0.4


What is important to analyze:
1. add new feature "prefer_discounts", make it bool
2. new feature "repeated purchases" - product_id what is repeatedly bought or NaN in case if customer doesn't have any
3. add feature "Family" according to the number of bought products, make it bool
4. add feature "wholesaler" if person make huge purchase seldom
